![NERLab](http://www.fee.unicamp.br/sites/default/files/docentes/leoelias/imagens/logo_v1%28WEB%29.png)
--

# NeMu: A neuromuscular model notebook

The present Python notebook (dubbed NeMu) encompasses a brief description and a mathematical model of the mechanisms underlying the generation of surface electromyogram (sEMG) and the muscle force during voluntary isometric contractions.

- Version 0.1: January 06, 2019;
- Version 0.2: July 22, 2019;
- Version 0.3: October 03, 2019;
- Version 0.4 | Last update: November 09, 2020.

Authors:  
[Ricardo Gonçalves Molinari](mailto:molinari@gmail.com) and [Leonardo Abdala Elias](mailto:leoelias@fee.unicamp.br)

# Import libraries and create model objects

In [16]:
if 'google.colab' in str(get_ipython()):
    !git clone -l -s https://github.com/molinaris/neuromuscular_notebook cloned-repo
    %cd cloned-repo
    !ls

from nerlab import  fug, emg, force, util
import ipywidgets as wi

util.config_plots()
mnpool = fug.Phemo()
muscle_emg = emg.Emg_mod(mnpool)
muscle_force = force.Muscle_force(mnpool)
sim = util.sim_results(mnpool, muscle_emg, muscle_force)

# The motor pool model
## Recruitment pattern of motor units

This section of the model generates the static input-output relationship of the motor pool. Here, we adopted the phenomenological model by [Fuglevand et al. (1993)](https://doi.org/10.1152/jn.1993.70.6.2470 "Models of recruitment and rate coding organization in motor-unit pools"), where the common drive to the motor neuron (MN) pool is mapped into the motor unit (MU) firing rate. A key aspect of this mapping is the recruitment pattern of MUs.

As well known, each MU encompasses a single MN and its innervated muscle fibers. In the present model, three types of MUs were represented:

- Type I (or S) MUs: low-threshold MUs with slowly contracting muscle units;
- Type IIa (or FR) MUs: these MUs have a higher recruitment threshold (as compared to type I MUs) and its muscle units are fast contracting but fatigue resistant;
- Type IIb (or FF) MUs: these MUs have the highest recruitment threshold with fast contracting and fatigable muscle units.

### Recruitment Range
___Recruitment range___  is used to define the recruitment threshold excitation ($RTE$) of each MU.

$$RTE_i  =exp \left ( \frac{ln(R_R)i}{n} \right )$$

where:
- $RTE_i$ is the recruitment threshold excitation of the MU $i$;
- $R_R$ is the recruitment range;
- $i$ is the MU index;
- $n$ is the total number of MUs.  


### Relationship between MU firing rate and the excitatory drive

The following equation is used to define the ***firing rate*** of each MU as a function of the ***excitatory (common) drive***.

$$\begin{align}
     if \quad e(t) \ge RTE_i , \quad  fr_i(t) &= g_{i}(e(t)-RTE_i) + MFR \\
     else \quad fr_i(t) & = 0\\
    \end{align}$$

where:
- $e(t)$ is the excitatory (common) drive as a function of time;
- $fr_i(t)$ is the mean firing rate of the $i$-th MU at time $t$; 
- $MFR$ is the minimum firing rate;
- $PFR_i$ is the peak firing rate of the $i$-th MU, defined by: 

$$ PFR_i   = PFR_1 - PFRD \frac{RTE_i}{RTE_n} $$
    
- $PFRD$ is the difference between the peak firing rate of the first and the last recruited MU of the pool;

- $g_{i}$ is the gain of the relationship between the excitatory drive and the firing rate for the $i$-th MU, defined by:

$$ \begin{align}
    g_{i} &= g_{f}(i) \frac{PFR_i - MFR}{e_{max} - RTE_i} \\
    e_{max} &= \frac{RTE_n}{\tilde{e}_{LR}} \\
\end{align}$$

where:
- $e_{max}$ is the maximum excitatory drive to the MN pool;
- $\tilde{e}_{LR}$ is the normalized (by $e_{max}$) excitatory drive that recruit the last MN.
- $g_{f}(i)$ is a function of $i$ that returns $n$ ($n$ = number of MNs) evenly spaced numbers over the interval defined by a gain factor $g_{1}$ and 1.

If the checkbox __Same gain for all MNs?__ is checked, $g_i = g_n$, resulting in the same gain for all MNs:

$$ g_{i} = \frac{PFR_n - MFR}{e_{max} - RTE_n} $$

For more information about the physiology of MUs, please refer to the monograph by [Kernell (2006)](https://doi.org/10.1093/acprof:oso/9780198526551.001.0001 "The Motoneurone and its Muscle Fibres").

In [19]:
ui1,ws1 = util.wi1()
out1 = wi.interactive_output(mnpool.view_organization, ws1)
display(wi.HBox([ui1,out1]))

## Definition of the excitatory (common) drive

The common drive and the recruitment pattern of the MUs (defined above) are used to generate the MU spike trains. These spike trains will command the muscle units so as to produce muscle force and electromyogram (EMG).

Different force intesities (percentage of the maximum force) may be explored with ***Trapezoidal*** or ***Sinusoidal*** commands. To simulate a steady isometric contraction, you can choose the ***Trapezoidal*** command, and then define ***Plateau on*** as zero and ***Plateau off*** as the final simulation time.

MU recruitment and de-recruitment can be observed using a ***Trapezoidal*** command, but in this scenario the value of the parameter ***Onset*** should be less than the ***Plateau on***, and ***Offset*** should be less than ***Plateau off***.

Periodic contractions can be simulated by changing the command to the ***Sinusoidal*** waveform, with the following parameters: ***Peak Intensity*** and ***Frequency***.

In [18]:
ws2,ui2 = util.wi2()
out2 = wi.interactive_output(mnpool.view_excitatory,ws2)
display(wi.HBox([ui2,out2]))

## Motor unit spike trains

The MU activity (spike trains) resembles a sthocastic point process with a gaussian distribution. The model by [Fuglevand et al. (1993)](https://doi.org/10.1152/jn.1993.70.6.2470 "Models of recruitment and rate coding organization in motor-unit pools") simulates the spike times as:

$$ t_{i,j}= \mu + \sigma. Z + t_{i,j-1}$$

where:
- $ t_{i,j}$ is the $j$-th spike time of the $i$-th MU;
- $ \mu $ is the mean interspike interval (ISI);
- $ \sigma $ is the standard deviation of each ISI, defined as a function of the coefficient of variation ($cv$) and $\mu$ ($\sigma = \mu . cv$);
- $ Z$ represents how the spike time deviates from the mean value (a random value between -3.9 and 3.9 from a gaussian distribution).  

### Motor unit synchronization

Convergent presynaptic commands to the spinal MNs promote common fluctuations on the membrane potentials of these cells. These common sources increases the probability of synchronous discharges of MUs (the so-called MU synchronization).

To simulate this phenomenon, we adopted the model by [Yao et al. 2000](https://doi.org/10.1152/jn.2000.83.1.441 "Motor-Unit Synchronization Increases EMG Amplitude and Decreases Force Steadiness of Simulated Contractions"). However, an extra step was added to the algorithm in order to deal with time varying excitations (trapezoidal and sinusoidal). The modification followed the proposal by [Shin et al. 2018](https://doi.org/10.1088/1741-2552/aa925d "Relative contribution of different altered motor unit control to muscle weakness in stroke: a simulation study"), where the synchronization process is limited to spike times ocurring in a maximum interval of 15 ms.

The ***Synch. Level [%]*** operates as follows:
1. For each MU in the pool, a percentage of the spike times are randomly picked up to serve as a reference for spike adjustment;
2. Given a reference spike, the algorithm selects a percentage of recruited MUs and finds the closest discharge to be adjusted (i.e., the discharge with the minimum distance between the reference spike and all other spikes from the selected MUs). 
3. The spike time selected in *step 2* is adjusted to the reference spike. A small amount of variability is imposed by the parameter ***Synch. $\sigma$ [ms]***, so the reference spike time is the mean value while ***Synch. $\sigma$ [ms]*** is the standard deviation of a gaussian distribution.

In [4]:
wi3 = util.wi3()
_ = wi.interact_manual(mnpool.view_neural_command, CoV = wi3[0], 
                       synch_level = wi3[1], sigma = wi3[2])

interactive(children=(FloatSlider(value=20.0, description='ISI $cv$ [%]:', layout=Layout(width='300px'), max=5…

# Surface electromyogram (sEMG)

The surface electromyogram (sEMG) emerges from the electric activity of the muscle fibers during a given contraction. Currents flowing across the sarcolemma (as a result of the muscle fiber action potential) produce an electric potential that can be recorded by extracellular electrodes. The recorded extracellular potentials are spatially filtered by the volume conductor, which includes the muscle, adipose, and skin tissues. It is worth noting that the MU territories are sparsely distributed in the muscle cross-section. Here, we represented the sEMG recorded by bipolar electrode (*red triangle* in the following graph) located on the skin.


## Muscle cross section morphology

Four different morphologies for the muscle cross section were included to represent the desired muscle. The following morphologies were included: ***Circle***, ***Ring***, ***Pizza***, and ***Ellipse***. Also, cross-section area (***CSA***), fat thickness (***Fat layer [mm]***), and skin thickness (***Skin layer [mm]***) are properties to be adjusted. For the ***Ring***, ***Pizza***, and ***Ellipse*** morphologies you need to adjust two additional parameters:

- ***Proportion*** defines, for the ***Ring*** morphology, the ratio between internal and external muscle radii and, for the ***Ellipse***, the ratio between semi-minor and semi-major axes;
- ***Theta [rad]*** defines the "opening" of the choosen geometry. In other words, the angle formed between the vertical line (which passes through the origin) and the line that defines the "side" boundary of the muscle ***CSA***, which also passes through the origin.

In [5]:
ws4,ui4 = util.wi4()
out4 = wi.interactive_output(muscle_emg.view_morpho, ws4)
display(wi.HBox([ui4, out4]))

## Motor unit distribution within the muscle CSA

Many human muscles present some level of MU regionalization within the muscle CSA ([Johnson et al. 1973](https://doi.org/10.1016/0022-510X(73)90023-3 "Data on the distribution of fibre types in thirty-six human muscles")). To simulate this MU regionalization, we defined two different distributions, one for MUs innervating type I muscle fibers and another for MUs innervating type II muscle fibers. Each MU position is defined using polar coordinates. All MUs will have their polar angles defined by a random variable with uniform distribution whose limits depend on the choosen morphology. The radial coordinate of each MU is also defined as a random variable, but with a gaussian distribution.

The distribution of the radial coordinate is adjusted by the variable ***Type I $\mu$*** (normalized by the muscle external radius), which defines the center (average) of the distribution for Type I MUs. Also, ***Type I $\sigma$*** (normalized by the muscle external radius) defines the standard deviation of the gaussian distribution of the radius for Type I MUs. Similarly, ***Type II $\mu$*** and ***Type II $\sigma$*** define the center (average) and the standard deviation (both normalized by the muscle external radius) of the distribution for Type II (a and b) MUs, respectively.

### Muscle fiber ratio

The MU territory area is highly correlated with the number of muscle fibers innervated by a single MN, which varies across the MU pool ([Enoka and Fuglevand 2001](https://doi.org/10.1002/1097-4598(200101)24:1<4::AID-MUS13>3.0.CO;2-F "Motor unit physiology: Some unresolved issues")). A typical skeletal muscle comprises many MUs with relatively few muscle fibers and few MUs with a large quantity of muscle fibers.  This MU innervation ratio along the motor pool can be modelled as:

$$y_i = y_1e^{\displaystyle \left( \frac{ln(R_{in})i}{n} \right)}$$

where:
- $y_i$ is the innervation number of the $i$-th MU;
- $y_1$ is the innervation number of the smallest MU;
- $n$ is the total number of MUs;
- $R_{in}$ is the ***Innervation ratio*** between the largest ($y_n$) and smallest ($y_1$) MU.

In [6]:
ui5,ws5 = util.wi5()
out5 = wi.interactive_output(muscle_emg.view_distribution, ws5)
display(wi.VBox([ui5, out5]))

## Motor unit action potentials (MUAPs)

MU action potentials (MUAPs) observed from bipolar electrodes at skin surface can be represented by 1st and 2nd order Hermite-Rodriguez (HR) functions ([Lo Conte et al. 1994](http://doi.org/10.1109/10.335863 "Hermite Expansions of Compact Support Waveforms: Applications to Myoelectric Signals")). These functions fit well the MUAPs shape and has been used in previous studies. The 1st-order HR function models biphasic MUAPs while the 2nd-order HR function models triphasic MUAPs. These functions are defined by the following equations:

$$\begin{align}
HR_1(t) &= \frac{A_M.(t-t_{AP})}{\lambda_M} e^{\displaystyle  - \left (\frac{t-t_{AP}}{\lambda_M} \right )^2}u(t-t_{AP}) \\
HR_2(t) &= A_M \left [1 - 2 \left ( \frac{t-t_{AP}}{\lambda_M} \right )^2 \right ] e^{\displaystyle - \left (\frac{t-t_{AP}}{\lambda_M} \right )^2} u(t-t_{AP})
\end{align}$$

where:
- $HR_k$ is the $k$-th order Hermite-Rodriguez (HR) function;
- $A_M$ is the amplitude factor;
- $\lambda_M$ is the duration factor;
- $t$ is simulation time;
- $t_{AP}$ is the MU discharge time;
- $u$ is the Heaviside (step) function.

For each MU, distinct amplitude and duration factors are attributed. To define each MUAP amplitude factor, an exponential interpolation between the ___First MUAP Amplitude [mV]___ and the ___Last MUAP Amplitude [mV]___ is used. The same method is adopted to define the duration factors of each MUAP.

In the present model, the MUAP can represented by either 1st-order or 2nd-order HR functions, which are randomly (equal probability) asigned to the MUs.

In [7]:
ui6,ws6 = util.wi6()
out6 = wi.interactive_output(muscle_emg.view_muap, ws6)
display(wi.HBox([ui6, out6]))

## Volume conductor (spatial filtering)

The volume conductor (characterized by muscle, fat, and skin tissues) filtering effect is modelled as isotropic and proportional to the distance between the MU territory center and the electrode on the skin surface. The the following equations ([Cisi and Kohn 2008](http://doi.org/10.1007/s10827-008-0092-8 "Simulation system of spinal cord motor nuclei and associated nerves and muscles, in a Web-based architecture")) were used to model the amplitude attenuation and MUAP widening caused by the spatial filtering of the volume conductor:

$$\begin{align}
V &= V_0 e^{ \left ( \frac{-d}{\tau_{at}} \right )} \\
T &= T_0(1+Cd)
\end{align} $$

where:
- $V$ is the MUAP amplitude recorded by the surface electrode;
- $V_0$ is the MUAP amplitude at the center of the MU territory;
- $d$ is the distance between the center of the MU territory and the center of the bipolar EMG sensor;
- $\tau_{at}$ is the ___Amplitude Attenuation Factor [$mm^{-1}$]___;
- $T$ is the MUAP duration at the electrode;
- $T_0$ is the MUAP duration at the center of the MU territory;
- $C$ is the MUAP ___Widening Factor [$mm^{-1}$]___.

In [8]:
ui7,ws7 = util.wi7()        
out7= wi.interactive_output(muscle_emg.view_attenuation,ws7)
display(ui7,out7)

Output()

## Generation of sEMG

The surface electromyogram (sEMG) is modelled as the summed activity of all filtered MUAP trains of recruited MUs.

To mimic experimental recordings, an additive gaussian noise can be summed to the sEMG signal generated by the model. The amplitude of this interference noise is adjusted by the ___Noise standard deviation [mV]___ parameter. Also, a 4th-order Butterworth bandpass filter was applied to the sEMG signal to represent the anti-aliasing filter present in EMG devices. The parameters ___High-pass cutoff frequency [Hz]___ and ___Low-pass cutoff frequency [Hz]___ define the lower and higher cutoff frequencies of the filter, respectively.

In [9]:
wi8 = util.wi8()
_ = wi.interact_manual(muscle_emg.view_semg, add_noise = wi8[0], noise_level = wi8[1], 
                       add_filter = wi8[4], bplc = wi8[2], bphc = wi8[3])

interactive(children=(Checkbox(value=True, description='Add Noise', layout=Layout(width='400px'), style=Descri…

## Analysis of sEMG signal

In [10]:
ui9,ws9 = util.wi9(muscle_emg)
out9 = wi.interactive_output(muscle_emg.analysis, ws9)
display(ui9, out9)

Output()

# Muscle force

## Motor unit twitch

MU twitch is the mechanical response of muscle fibers to action potentials arriving at the neuromuscular junction. Under isometric conditions, muscle twitch can be modelled as the impulse response of a second-order critically damped system  ([Milner-brown et al. 1973](https://doi.org/10.1113/jphysiol.1973.sp010087 "The contractile properties of human motor units during voluntary isometric contractions")). The general formula for the impulse response of a second-order critically damped system is:

$$ F(t) = P \frac{t}{T}e^{\displaystyle \left( \frac{-t}{T} \right)}u(t) \; , \quad t \in \Re$$

where:
- $F(t)$ is the MU twitch force;
- $P$ is the twitch amplitude;
- $T$ is the time to peak;
- $u(t)$ is the Heaviside (step) function.

The twitch amplitude is related to the MU size and the MU recruitment threshold. This relationship can be modelled by the following equation ([Fuglevand et al. 1993](https://doi.org/10.1152/jn.1993.70.6.2470 "Models of recruitment and rate coding organization in motor-unit pools")):

$$ P_i = P_1e^{\displaystyle \left( \frac{ln(R_P)i}{n}\right) }$$

where:
- $P_i$ is the peak twitch force of the $i$-th MU;
- $P_1$ is the peak twitch force of the first MU;
- $R_P$ is the the range of twitch forces;
- $n$ is the number of MUs in the pool.

The range of twitch time-to-peak along the MU pool can be modelled by an exponential function, as proposed in the original paper by [Fuglevand et al. 1993](https://doi.org/10.1152/jn.1993.70.6.2470 "Models of recruitment and rate coding organization in motor-unit pools") (see the equation below).

$$ T_i=T_L \left ( \frac{1}{P_i} \right )^{\displaystyle \left( \frac{1}{log_{R_T}(R_P)} \right)} $$

where:
- $T_i$ is the twitch time-to-peak of the $i$-th MU;
- $T_L$ is the largest $T_i$;
- $R_T$ is the desired range of $T_i$ for the MU pool.

In some human muscles there is no evident relation between the twitch time-to-peak and the MU recruitment threshold ([Bigland-Ritche et al. 1998](http:doi.org/10.1177/107385849800400413 "Contractile properties of human motor units: Is man a cat?")). This condition can be simulated here by checking the button `Random uniform`, so that the MU twitch time-to-peak is uncorrelated with the twitch amplitude. The time-to-peak of each MU will be uniformly distributed between the interval defined by $T_L/R_T$ and $T_L$.

In [11]:
ui10,ws10 = util.wi10()
out10 = wi.interactive_output(muscle_force.view_mu_twitch, ws10)
display(wi.HBox([ui10,out10]))

## Motor unit tetanic force

The MU behavior has several nonlinear properties, including the relation between the MU isometric force and the stimulus frequency ([Milner-brown et al. 1973](https://doi.org/10.1113/jphysiol.1973.sp010087 "The contractile properties of human motor units during voluntary isometric contractions")). The so-called force-frequency relationship has a sigmoidal shape, so that the force of the MU saturates at high-frequency stimulation. Typically, the stimulus rate for MU force saturation is higher than 50 imp/s for muscles that control the digits of the hand ([Fuglevand et al 1999](https://doi.org/10.1152/jn.1999.81.4.1718 "Force-Frequency and Fatigue Properties of Motor Units in Muscles That Control Digits of the Human Hand")). Also, the shape of the force-frequency relationship depends on the contractile properties of the MU. Fast-contracting MUs tend to fuse faster when compared with slowly-contracting MUs. It is also important to note that the stimulus rate for MU force saturation is much higher than the firing rate produced by the MU during a maximum voluntary contraction (mean rate of ~35 imp/s for the first dorsal interosseous muscle, [Enoka and Fuglevand 2001](https://doi.org/https://doi.org/10.1002/1097-4598(200101)24:1<4::AID-MUS13>3.0.CO;2-F "Motor unit physiology: Some unresolved issues:")).

To simulate the saturation mechanisms, i.e., the nonlinear relationship between stimulus rate and MU force, we used the following function ([Watanabe et al. 2013](https://doi.org/10.1152/jn.00073.2013 "Influences of premotoneuronal command statistics on the scaling of motor output variability during isometric plantar flexion")):

$$ F_{sat} (t)=\frac{1-e^{-cF(t)}}{1+e^{-cF(t)}} $$

To find the parameter $c$, we first stimulate the MU at a rate that produces the desired $F_{sat}$ of the MU. After, the MU is stimulated with a single pulse to produce a unitary twitch amplitude ($P=1$). Then, we use the secant method (root-finding algorithm with secant lines and with better approximations at each step) to find the value of $c$ that solve the following equation:

$$max(F_{sat}(t)) - 0.999 = 0$$

In [12]:
ui11,ws11 = util.wi11(muscle_force)
out11 = wi.interactive_output(muscle_force.view_saturation, ws11)
display(ui11, out11)

Output()

## Generation of muscle force

At this point, all force parameters are defined and the simulation is ready to be performed (press the `Run Interact` button). To speed up simulations, we used a digital filter to produce the twitch time course described above. The spike train was modelled as a train of Kronecker deltas:

$$ s(m) = \sum_j \delta(m-m_j) \; , \quad m \in Z$$

where:
- $m_j$ is the $j$-th MU spike sample;
- $\delta(m)$ is the Kronecker delta function.

The discrete-time version of the twitch model was obtained by the impulse invariance technique, where the z-transform  was used to obtain the difference equation (see below) that describes the linear part of the force developed by the MU ([Cisi and Kohn 2008](https://doi.org/10.1007/s10827-008-0092-8 "Simulation system of spinal cord motor nuclei and associated nerves and muscles, in a Web-based architecture.")).

$$     \begin{split}
F_{MU_i}(m) =  2 \, e^{\displaystyle \left( \frac{- \Delta t}{T_{i}} \right )} \, F_{MU_i} (m-1) \, -e^{\displaystyle \left ( \frac{-2 \Delta t}{T_i} \right )}F_{MU_i}(m-2)   \\
 + \frac{\displaystyle P_i \, \Delta t^2}{\displaystyle T_i} e^{\displaystyle \left (  1- \frac{\Delta t}{T_i} \right )} s(m-1)
    \end{split} $$

where:
- $F_{MU_i}(m)$ is the force of the $i$-th MU at sample $m$;
- $\Delta t$ is the integration time step;

The last step during the MU force generation is the saturation mechanism described previously. Finally, the whole muscle force is calculated as the sum of all MU forces.

In [13]:
wif = wi.interact_manual(muscle_force.view_force)

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

## Analysis of muscle force signal

In [14]:
ui12, ws12 = util.wi12(muscle_force)
out12 = wi.interactive_output(muscle_force.analysis, ws12)
display(ui12, out12)

Output()

# Saving simulation results

The configuration used to simulate this neuromuscular system and its results can be saved in the following folder: `\simulation_results\folder_name`. The ***folder name*** can be selected in the following cell. To save it, just click on **Run interact** button.


In [15]:
ws13 = util.wi13()
_ = wi.interact_manual(sim.save_results, 
                       save_conf = ws13[0], save_spikes = ws13[1], 
                       save_emg  = ws13[2], save_force  = ws13[3],
                       save_dist = ws13[4], save_muap   = ws13[5], 
                       save_CT   = ws13[6], folder_name = ws13[7])

interactive(children=(Checkbox(value=True, description='Save simulation config.'), Checkbox(value=True, descri…